In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from pymongo import MongoClient
import configparser
import logging
import logging.config
sys.path.append(os.path.join(os.path.dirname('__file__'),'..'))
sys.path.append('/Users/macico/Dropbox/btc/commons')
os.environ['KULOKO_DIR'] = '/Users/macico/Dropbox/btc/kuloko/kuloko_handler'
os.environ['MONGO_DIR'] = '/Users/macico/Dropbox/btc/commons/mongodb'
os.environ['LOGDIR'] = '/Users/macico/Dropbox/btc/kuloko/log'
KULOKO_DIR=os.environ['KULOKO_DIR'] 
MONGO_DIR=os.environ['MONGO_DIR'] 
LOGDIR=os.environ['LOGDIR'] 

logging.config.fileConfig(os.path.join(KULOKO_DIR,'ini/logconfig.ini'),defaults={'logfilename': os.path.join(LOGDIR,'logging.log')})
logger = logging.getLogger("KULOKO")


In [3]:
mongo_ini = configparser.ConfigParser()
mongo_ini.read(os.path.join(MONGO_DIR,'ini/mongo_config.ini'), encoding='utf-8')

from mongodb.src.mongo_handler import MongoHandler

In [4]:

import  kuloko_handler.handler.socket_handler as skt_api

private_api_ini = configparser.ConfigParser()
private_api_ini.read(os.path.join(KULOKO_DIR,'ini/private_api.ini'), encoding='utf-8')

general_config_ini = configparser.ConfigParser()
general_config_ini.read(os.path.join(KULOKO_DIR,'ini/config.ini'), encoding='utf-8')


['/Users/macico/Dropbox/btc/kuloko/kuloko_handler/ini/config.ini']

In [9]:
orderbooks = skt_api.Orderbooks(logger, general_config_ini,private_api_ini)
orderbooks.connect(orderbooks.get_public_socket_url(),'BTC')
orderbooks.subscribe()
mongo_db = MongoHandler(mongo_ini['LOCAL'],"orderbook")

In [11]:
import time
retry_cnt=0
max_retry_cnt=60
print("START")
while True:
    try:
        time.sleep(10)
        data = orderbooks.get()
        print("Data:{0}, Retry_cnt={1}".format(len(data),retry_cnt))
        if len(data)==0:
            retry_cnt += 1
            continue
        insert_json =[ orderbooks.convert_shape(_data,5,"json") for _data in data ]
        mongo_db.insert_many(insert_json)
        retry_cnt = 0
    except Exception as e:
        retry_cnt += 1
        if retry_cnt < max_retry_cnt:
            continue
        print("END",e)
        break


START
Data:50, Retry_cnt=0
Data:20, Retry_cnt=0
Data:21, Retry_cnt=0
Data:19, Retry_cnt=0
send: b'\x8a\x80\xc7\xc7\x086'
Data:20, Retry_cnt=0
Data:19, Retry_cnt=0
Data:20, Retry_cnt=0
Data:20, Retry_cnt=0


In [34]:
orderbooks.disconnect()

In [18]:
orderbooks.disconnect()

In [19]:
data = orderbooks.get()

In [20]:
data

005'},
   {'price': '1993800', 'size': '0.0068'},
   {'price': '1994000', 'size': '0.0504'},
   {'price': '1994050', 'size': '0.0054'},
   {'price': '1994118', 'size': '0.0051'},
   {'price': '1995000', 'size': '0.027'},
   {'price': '1995099', 'size': '0.0551'},
   {'price': '1995295', 'size': '0.0051'},
   {'price': '1998455', 'size': '0.004'},
   {'price': '1998750', 'size': '0.25'},
   {'price': '1999000', 'size': '0.1'},
   {'price': '1999804', 'size': '0.0051'}],
  'bids': [{'price': '1988600', 'size': '0.1'},
   {'price': '1988590', 'size': '0.1'},
   {'price': '1988585', 'size': '0.4'},
   {'price': '1987000', 'size': '0.003'},
   {'price': '1986627', 'size': '0.1'},
   {'price': '1986100', 'size': '0.5'},
   {'price': '1986079', 'size': '0.2'},
   {'price': '1985662', 'size': '0.2219'},
   {'price': '1985400', 'size': '1.2378'},
   {'price': '1985050', 'size': '2.5'},
   {'price': '1985000', 'size': '2.5'},
   {'price': '1984850', 'size': '2.5'},
   {'price': '1984750', 'size'